
# 🌾 Grain Recommender Agent — Tutorial Runner (ipynb)

이 노트북은 **기존 폴더 구조**(예: `grain_recommender_agent/`)를 그대로 사용해  
**코드가 보이게 동작**하는 최소 실행 튜토리얼입니다.

> ▶️ 목적: _정식 앱 배포 없이_ **노트북에서** `Agent(Plan & Schedule)` 흐름을 1회 실행하고,  
> 원하시면 **FastAPI도 노트북 안에서 임시로 띄워** `/recommend`를 호출해볼 수 있게 구성했습니다.



## 0) 사전 준비
- 이미 만들어둔 루트 폴더를 지정하세요. (Windows 예시)
  - `C:\Users\hslee\Desktop\grain_recommender_agent`
- 이 노트북은 **파일을 덮어쓰지 않습니다.**
  - *빈 파일(크기 0)*인 경우에만 **데모용 최소 코드**를 주입합니다. (주입 여부는 선택 가능)


In [ ]:

# <<< 사용자 편집 구역 >>>
PROJECT_ROOT = r"C:/Users/hslee/Desktop/grain_recommender_agent"  # ← 본인 경로로 변경
assert PROJECT_ROOT and isinstance(PROJECT_ROOT, str)
print("PROJECT_ROOT =", PROJECT_ROOT)


In [ ]:

import os, sys, pathlib, json, time

# 폴더 존재 확인
proj = pathlib.Path(PROJECT_ROOT)
assert proj.exists(), f"경로 없음: {proj}"

# 간단 트리 출력
for base, dirs, files in os.walk(proj):
    rel = base.replace(str(proj), ".")
    print(rel if rel else ".", "/")
    for d in sorted(dirs):
        print("  [D]", d)
    for f in sorted(files):
        print("  [F]", f)



## 1) 패키지 인식 설정 (필요 시)
- `app/` 폴더를 모듈로 임포트하려면 `__init__.py`가 있어야 합니다.
- 아래 셀은 **없으면 만들어 주는** 유틸입니다.


In [ ]:

import pathlib

def ensure_init_py(dirpath: str):
    p = pathlib.Path(dirpath) / "__init__.py"
    if not p.exists():
        p.write_text("# package marker\n", encoding="utf-8")
        print("created:", p)
    else:
        print("ok:", p)

for sub in ["app", "app/agent", "app/tools", "app/services", "app/api", "app/schemas", "app/scripts", "app/tests"]:
    ensure_init_py(str(pathlib.Path(PROJECT_ROOT) / sub))



## 2) (선택) **빈 파일에만** 데모 최소 코드 주입
- *파일 크기가 0인 경우에만* 스켈레톤 코드를 기록합니다. (덮어쓰기 방지)
- 이미 구현하신 파일이 있다면 **건드리지 않습니다**.
- 주입 대상(최소): `state.py`, `extractor.py`, `validator.py`, `rules_engine.py`, `formatter.py`, `graph.py`, `api/main.py`, `api/models.py`


In [ ]:

import os, pathlib

STATE_SRC = "\n\"\"\"\nDEMO-ONLY: Minimal Agent state models (dict-less for simplicity).\nReplace with Pydantic in production.\n\"\"\"\nfrom typing import Dict, Any, List, Optional\nfrom dataclasses import dataclass, field\n\n@dataclass\nclass AgentState:\n    user_id: str\n    input_text: Optional[str] = None\n    survey: Dict[str, Any] = field(default_factory=dict)\n    validation: Optional[Dict[str, Any]] = None\n    primary: Optional[Dict[str, Any]] = None\n    candidates: List[Dict[str, Any]] = field(default_factory=list)\n    final: Optional[Dict[str, Any]] = None\n    memory: Dict[str, Any] = field(default_factory=lambda: {\"liked\": [], \"disliked\": [], \"recent_feedback\": None})\n"
EXTRACTOR_SRC = "\n# DEMO-ONLY extractor: simple keyword-based parser.\nimport re\n\ndef run(state):\n    text = (state.input_text or \"\").lower()\n    survey = dict(state.survey)\n\n    if any(k in text for k in [\"\ud608\ub2f9\", \"diabetes\"]):\n        survey[\"purpose\"] = {\"value\": \"\ud608\ub2f9\uad00\ub9ac\", \"confidence\": 0.9, \"source\": \"extractor\"}\n        survey[\"health_issue\"] = {\"value\": \"diabetes\", \"confidence\": 0.8, \"source\": \"extractor\"}\n    elif any(k in text for k in [\"\ub2e4\uc774\uc5b4\ud2b8\", \"\uccb4\uc911\"]):\n        survey[\"purpose\"] = {\"value\": \"\uccb4\uc911\uad00\ub9ac\", \"confidence\": 0.9, \"source\": \"extractor\"}\n\n    m = re.search(r\"\uc8fc\\s*(\\d+)\ub07c\", text)\n    if m:\n        survey[\"frequency\"] = {\"value\": int(m.group(1)), \"confidence\": 0.8, \"source\": \"extractor\"}\n\n    if \"\ucc30\uc9c4\" in text:\n        survey[\"texture_pref\"] = {\"value\": \"\ucc30\uc9c4\ubc25\", \"confidence\": 0.8, \"source\": \"extractor\"}\n    elif \"\uace0\uc2ac\" in text:\n        survey[\"texture_pref\"] = {\"value\": \"\uace0\uc2ac\ubc25\", \"confidence\": 0.8, \"source\": \"extractor\"}\n\n    if \"\ubcf4\ub9ac\" in text and (\"\uc54c\ub808\ub974\uae30\" in text or \"\ubabb \uba39\" in text or \"\ud53c\ud568\" in text):\n        disliked = set(state.memory.get(\"disliked\", []))\n        disliked.add(\"\ubcf4\ub9ac\")\n        state.memory[\"disliked\"] = sorted(list(disliked))\n\n    return {\"survey\": survey}\n"
VALIDATOR_SRC = "\n# DEMO-ONLY validator: required-only\nREQUIRED = [\"purpose\", \"frequency\", \"texture_pref\"]\nHIGH_WEIGHT = [\"purpose\", \"health_issue\"]\n\ndef run(state):\n    survey = state.survey or {}\n    missing = [k for k in REQUIRED if k not in survey or survey[k].get(\"value\") in (None, \"\", [])]\n    conflicts = []\n\n    if survey.get(\"health_issue\", {}).get(\"value\") == \"diabetes\":\n        if survey.get(\"gi_pref\", {}).get(\"value\") == \"high\":\n            conflicts.append(\"diabetes vs high GI\")\n\n    ok = (len(missing) == 0 and len(conflicts) == 0)\n    return {\n        \"validation\": {\n            \"ok\": ok,\n            \"missing_required\": missing,\n            \"conflicts\": conflicts,\n            \"high_weight_missing\": [k for k in HIGH_WEIGHT if k in missing]\n        }\n    }\n"
RULES_SRC = "\n# DEMO-ONLY rules engine\nGRAINS = {\n    \"\ubc31\ubbf8\": {\"gi\": 70},\n    \"\ud604\ubbf8\": {\"gi\": 55},\n    \"\uadc0\ub9ac\": {\"gi\": 55},\n    \"\ubcf4\ub9ac\": {\"gi\": 50},\n    \"\uba54\ubc00\": {\"gi\": 54},\n}\nDEFAULT_RATIO = {\"\ubc31\ubbf8\": 50, \"\ud604\ubbf8\": 30, \"\uadc0\ub9ac\": 20}\nGI_CAP = 65\n\ndef _normalize(mix):\n    tot = sum(mix.values()) or 1.0\n    return {k: round(v * 100 / tot, 1) for k, v in mix.items()}\n\ndef run(state):\n    survey = state.survey or {}\n    disliked = set(state.memory.get(\"disliked\", []))\n    base = dict(DEFAULT_RATIO)\n\n    for g in list(base.keys()):\n        if g in disliked:\n            del base[g]\n\n    if survey.get(\"purpose\", {}).get(\"value\") == \"\ud608\ub2f9\uad00\ub9ac\":\n        if \"\ubc31\ubbf8\" in base:\n            base[\"\ubc31\ubbf8\"] = max(0, base[\"\ubc31\ubbf8\"] - 20)\n        for cand in [\"\ud604\ubbf8\", \"\uadc0\ub9ac\", \"\ubcf4\ub9ac\", \"\uba54\ubc00\"]:\n            if cand not in disliked:\n                base[cand] = base.get(cand, 0) + 10\n\n    for g in list(base.keys()):\n        if GRAINS.get(g, {}).get(\"gi\", 0) > GI_CAP:\n            base[g] = max(0, base[g] - 10)\n            for alt in [\"\ud604\ubbf8\", \"\uadc0\ub9ac\", \"\ubcf4\ub9ac\", \"\uba54\ubc00\"]:\n                if alt not in disliked:\n                    base[alt] = base.get(alt, 0) + 10\n                    break\n\n    mix = _normalize(base)\n    score = round(sum((1.0 - GRAINS[g][\"gi\"]/100.0) * v for g, v in mix.items() if g in GRAINS)/100, 3)\n    return {\"primary\": {\"id\": \"R0\", \"mix\": mix, \"score\": score}, \"candidates\": []}\n"
FORMATTER_SRC = "\n# DEMO-ONLY formatter\ndef run(state):\n    p = state.primary or {}\n    lines = [\"\u2705 \ucd94\ucc9c \ubc30\ud569\uc548(R0):\", \"- \" + \", \".join([f\"{g} {r}%\" for g, r in p.get(\"mix\", {}).items()])]\n    text = \"\\n\".join(lines)\n    return {\"final\": {\"text\": text, \"payload\": {\"primary\": p, \"extras\": []}}}\n"
GRAPH_SRC = "\n# DEMO-ONLY graph (sequential)\nfrom .state import AgentState\nfrom ..tools import extractor, validator, rules_engine, formatter\n\ndef build_graph():\n    def invoke(state: AgentState) -> AgentState:\n        # extractor\n        state.survey = {**state.survey, **extractor.run(state).get(\"survey\", {})}\n        # validator\n        state.validation = validator.run(state).get(\"validation\", {})\n        if not state.validation.get(\"ok\"):\n            missing = \", \".join(state.validation.get(\"missing_required\", []))\n            state.final = {\"text\": f\"\ucd94\uac00 \uc815\ubcf4\uac00 \ud544\uc694\ud569\ub2c8\ub2e4: {missing}\", \"payload\": {\"type\": \"reask\", \"missing\": state.validation.get(\"missing_required\", [])}}\n            return state\n        # rules\n        out = rules_engine.run(state)\n        state.primary = out.get(\"primary\")\n        state.candidates = out.get(\"candidates\", [])\n        # format\n        state.final = formatter.run(state).get(\"final\")\n        return state\n    return type(\"Graph\", (), {\"invoke\": staticmethod(invoke)})\n"
API_MODELS_SRC = "\n# DEMO-ONLY FastAPI models\nfrom pydantic import BaseModel\nfrom typing import Optional, Dict, Any\n\nclass RecommendIn(BaseModel):\n    user_id: str\n    text: Optional[str] = None\n\nclass RecommendOut(BaseModel):\n    message: str\n    payload: Dict[str, Any]\n"
API_MAIN_SRC = "\n# DEMO-ONLY FastAPI app\nfrom fastapi import FastAPI\nfrom .models import RecommendIn, RecommendOut\nfrom ..agent.state import AgentState\nfrom ..agent.graph import build_graph\n\napp = FastAPI(title=\"Grain Recommender Agent (Demo)\")\n_graph = build_graph()\n\n@app.post(\"/recommend\", response_model=RecommendOut)\ndef recommend(inp: RecommendIn):\n    state = AgentState(user_id=inp.user_id, input_text=inp.text)\n    out = _graph.invoke(state)\n    final = out.final or {\"text\": \"\", \"payload\": {}}\n    return RecommendOut(message=final[\"text\"], payload=final[\"payload\"])\n"

FILES = [
    ("app/agent/state.py", STATE_SRC),
    ("app/tools/extractor.py", EXTRACTOR_SRC),
    ("app/tools/validator.py", VALIDATOR_SRC),
    ("app/tools/rules_engine.py", RULES_SRC),
    ("app/tools/formatter.py", FORMATTER_SRC),
    ("app/agent/graph.py", GRAPH_SRC),
    ("app/api/models.py", API_MODELS_SRC),
    ("app/api/main.py", API_MAIN_SRC),
]

def write_if_empty(root, relpath, content):
    p = pathlib.Path(root) / relpath
    if not p.exists():
        p.parent.mkdir(parents=True, exist_ok=True)
        p.write_text(content, encoding="utf-8")
        print("created (new):", p)
        return True
    else:
        if p.stat().st_size == 0:
            p.write_text(content, encoding="utf-8")
            print("filled (empty):", p)
            return True
        else:
            print("skip (non-empty):", p)
            return False

changed = 0
for rel, content in FILES:
    if write_if_empty(PROJECT_ROOT, rel, content):
        changed += 1

print("총", changed, "개 파일에 데모 코드를 기록했습니다. (나머지는 건드리지 않음)")



## 3) PYTHONPATH 설정 & 모듈 임포트
- 노트북에서 프로젝트 모듈을 import하려면 `sys.path`에 루트를 추가합니다.


In [ ]:

import sys, pathlib
root = pathlib.Path(PROJECT_ROOT)
if str(root) not in sys.path:
    sys.path.insert(0, str(root))
print("sys.path head:", sys.path[:3])

from app.agent.state import AgentState
from app.agent.graph import build_graph
print("Import OK.")



## 4) **직접 그래프 실행** (노트북 내에서 1회 흐름)


In [ ]:

graph = build_graph()
state = AgentState(user_id="user-001", input_text="혈당 관리가 필요하고 주 6끼 먹어요. 찰진밥 선호, 보리 알레르기 있어요.")
out = graph.invoke(state)
print(out.final["text"] if out.final else "(no final)")
out.final



## 5) (옵션) FastAPI 서버를 노트북에서 임시로 띄우기
> 로컬 환경에 `fastapi`, `uvicorn`, `pydantic`이 설치되어 있어야 합니다.


In [ ]:

import threading, time

def run_uvicorn():
    import uvicorn
    from app.api.main import app
    uvicorn.run(app, host="127.0.0.1", port=8000, log_level="info")

t = threading.Thread(target=run_uvicorn, daemon=True)
t.start()
time.sleep(2)
print("Uvicorn thread started on http://127.0.0.1:8000")



## 6) (옵션) HTTP 호출 테스트


In [ ]:

import requests, json
url = "http://127.0.0.1:8000/recommend"
payload = {"user_id": "user-002", "text": "다이어트 목표, 주 7끼, 고슬밥 선호"}
res = requests.post(url, json=payload, timeout=10)
print("Status:", res.status_code)
res.json()
